In [1]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
from gezi.common import *
from src.config import *
from src.preprocess import *
from src.postprocess import *
from src.eval import *
gezi.init_flags()

In [2]:

V = 16
models = [
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5',
  'deberta-v3-base.len1280.flag-encode_targets-cls-crank.lr-5e-5.a',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5',
  'deberta-v3.len1280.flag-encode_targets-cls-crank',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.ft2',
  'deberta-v3.len1280.flag-encode_targets-cls',
  
  'deberta-v3.len1280.flag-encode_targets-cls.v1_only',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.v1_ft.v1_dynamic',
  'deberta-v3-base.len1280.flag-encode_targets-cls.lr-5e-5.v1_only',
   
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-nl',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-de',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-pt',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-af',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cn',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ru',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-fi',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-sv',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ja',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ko',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-el',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-hr',
  'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cy',
  
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-cs',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-be',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-ar',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-es',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-it',
  # 'deberta-v3.len1280.flag-encode_targets-cls-crank.pre-trans-da',
]
len(models)

23

In [3]:
oofs = [gezi.read_oof(model, V, return_dict=False) for model in models]

In [4]:
oofs[0]

,text_id,label,n_words,pred,fold
0,0022683E9EA5,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",533,"[2.97, 2.682, 2.98, 2.756, 2.488, 2.639]",0
1,004AC288D833,"[3.5, 4.0, 4.0, 3.5, 3.5, 4.0]",328,"[3.156, 3.076, 3.232, 3.06, 3.098, 3.281]",0
2,01501F95B8B2,"[3.0, 4.0, 3.0, 4.0, 3.5, 3.0]",676,"[3.646, 3.643, 3.754, 3.84, 3.678, 3.434]",0
3,017D2125A1DE,"[2.5, 2.5, 3.0, 2.0, 2.0, 2.5]",379,"[2.38, 2.309, 2.812, 2.375, 2.266, 2.377]",0
4,0230A077F5B6,"[3.0, 2.5, 2.5, 2.5, 2.0, 2.5]",536,"[2.95, 2.63, 2.914, 2.666, 2.432, 2.344]",0
...,...,...,...,...,...
3906,FF7FE6BD8509,"[3.5, 3.5, 3.5, 3.5, 3.5, 3.5]",622,"[3.484, 3.469, 3.621, 3.658, 3.676, 3.281]",4
3907,FFA6690BC309,"[4.0, 3.5, 4.0, 3.5, 3.5, 4.0]",624,"[3.672, 3.271, 3.482, 3.39, 3.09, 3.506]",4
3908,FFAEAF8D0C90,"[2.5, 2.0, 2.5, 1.5, 2.0, 2.0]",496,"[2.758, 2.387, 2.723, 2.354, 2.174, 2.225]",4
3909,FFCDB2524616,"[2.5, 3.0, 3.0, 4.0, 3.5, 3.0]",263,"[3.262, 3.324, 3.477, 3.559, 3.693, 3.344]",4


In [5]:
# svr_oof = oofs[0].copy()

In [6]:
# svr_pred = np.load('../working/svr.npy')
# svr_oof['pred'] = svr_pred

In [7]:
ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
for i, oof in enumerate(oofs):
  if i == 0:
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i == 1: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float) * 2)
  elif i == 2: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.25)
  elif i == 3: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float))
  elif i == 4: 
    ensembler.add(oof, weight=np.array([2,1,10,1,1,2], float) * 2)
  elif i == 5: 
    ensembler.add(oof, weight=np.array([1,1,1,1,10,1], float) * 0.25)
  elif i == 6:
    ensembler.add(oof, weight=np.array([1,1,10,10,10,1], float))
  elif i == 7: 
    ensembler.add(oof, weight=np.array([1,1,10,1,1,1], float))
  elif i == 8: 
    ensembler.add(oof, weight=np.array([1,1,1,1,1,1], float) * 0.5)
  else:
    ensembler.add(oof, weight=np.array([1,1,10,1,0.25,0.25], float) * 0.8)

x = ensembler.finalize()
# ensembler2 = gezi.Ensembler(includes=['pred'], inplace=False)
# ensembler2.add(x, weight=np.array([1,1,1,1,1,1], float))
# # ensembler2.add(svr_oof, weight=np.array([1,1,1,1,1,1], float))
# x = ensembler2.finalize()

In [8]:
x['label'] = x.label.apply(list)
x['pred'] = x.pred.apply(list)

In [9]:
m = calc_metrics(np.array(list(x.label.values)), np.array(list(x.pred.values)))
m

OrderedDict([('score', 0.44514639359927893),
             ('score/cohesion', 0.47147124711971033),
             ('score/syntax', 0.4401423006640847),
             ('score/vocabulary', 0.40854637655346476),
             ('score/phraseology', 0.4508095468407472),
             ('score/grammar', 0.46257466576340794),
             ('score/conventions', 0.4373342246542586),
             ('score/cls', 0.46699584509031783),
             ('acc', 0.42482740986959855),
             ('acc/cohesion', 0.3889030938378931),
             ('acc/syntax', 0.42546663257478906),
             ('acc/vocabulary', 0.4622858603937612),
             ('acc/phraseology', 0.4231654308361033),
             ('acc/grammar', 0.4142163129634365),
             ('acc/conventions', 0.43492712861160826),
             ('score/max', 0.5656777675244553),
             ('acc/max', 0.285349015597034),
             ('score/min', 0.3727645085295607),
             ('acc/min', 0.4942469956532856),
             ('score/max2', 0.4447415

In [10]:
calc_metric(np.array(list(x.label.values))[:,0], np.array(list(x.pred.values))[:,0])

0.47147124711971033

In [11]:
import optuna
from optuna import Trial

In [17]:
idx = 9
def deal(i, pseudo=True, min_val=1, max_val=5):
  models = len(oofs)
  target = TARGETS[i]
  ic(i, target, m[f'score/{target}'])
  def objective(trial):
    ensembler = gezi.Ensembler(includes=['pred'], inplace=False)
    weights = [0] * models
    # w = trial.suggest_int(str(idx), min_val, max_val)
    w = trial.suggest_float(str(idx), 0, 1)
    for j in range(models):
      if j < idx:
        if pseudo or (not j in [6, 7, 8]):
          # weights[j] = trial.suggest_int(str(j), min_val, max_val)
          weights[j] = trial.suggest_float(str(j), 0, 1)
      else:
        weights[j] = w
    for j, oof in enumerate(oofs):
      weight = np.array([1,1,1,1,1,1], float)
      weight[i] = weights[j]
      ensembler.add(oof, weight=weight)
    x = ensembler.finalize()
    x['label'] = x.label.apply(list)
    x['pred'] = x.pred.apply(list)
    score = calc_metric(np.array(list(x.label.values))[:,i], np.array(list(x.pred.values))[:,i])
    return score
  study = optuna.create_study(direction='minimize')
  # study.optimize(objective, n_trials=FLAGS.n_trials, callbacks=[wandbc])
  n_trials = 500
  study.optimize(objective, n_trials=n_trials)
  return study

In [21]:
study = deal(4)

[11/30/22 02:26:11] 2696076698.py:5 in deal()
                    i: 4
                    target: 'grammar'
                    m[f'score/{target}']: 0.46257466576340794
[I 2022-11-30 02:26:11,170] A new study created in memory with name: no-name-f9c0bf37-50e5-491b-bdd5-a6583262f6bf
[I 2022-11-30 02:26:11,411] Trial 0 finished with value: 0.46382995651063486 and parameters: {'9': 0.8321788249239638, '0': 0.724540640904041, '1': 0.017979470412823773, '2': 0.6888070053278581, '3': 0.5251080744458495, '4': 0.6624614574733697, '5': 0.7417132220387342, '6': 0.6408568838172669, '7': 0.7553514315496546, '8': 0.841988482765139}. Best is trial 0 with value: 0.46382995651063486.
[I 2022-11-30 02:26:11,649] Trial 1 finished with value: 0.4633910141484206 and parameters: {'9': 0.15947139809465805, '0': 0.8088673170767048, '1': 0.6563128475031368, '2': 0.17507095492044256, '3': 0.7376849654685157, '4': 0.7632523819221472, '5': 0.3220799707453633, '6': 0.16710848244043597, '7': 0.26084076598716677,

In [ ]:
l = [0] * (idx + 1)
for i in range(idx + 1):
 l[i] = study.best_params[str(i)]
l

[0.3229300125723386,
 0.00012050160841757594,
 0.1722532849751525,
 0.0221457610346725,
 0.033184871657134615,
 0.05636443828099963,
 0.9992723377683341,
 0.047546132343093864,
 0.397917135506078,
 0.11443578411776703]

In [16]:
weights = [
  [0, 3, 2, 0, 9, 1, 3, 9, 0, 2],
  
]